In [15]:
from deap import base, creator, tools, algorithms
import random
from functools import reduce


INDS_LEN = 4
NUM_LIMIT = 3
SUM_CONSTRAINT = 100


def evaluate(individual):
    return reduce(lambda x, y: x * (y if y != 0 else 1), individual, 1), 

def feasible(inidividual):
    f1 = sum(inidividual) == SUM_CONSTRAINT
    
    i = 0
    for ind in inidividual:
        if ind != 0:
            i += 1
    f2 = i == NUM_LIMIT
    
    return f1 and f2

def repair(individual):
    """ 
    the current logic is that after crossover/mutation, i still filter out the top larget elements
    but the question is that it means i believe the larger the better? but not always
    so the solution should be that just randomly pickup N element that is positive and N is the element
    amount constraint
    """ 
    non_zero_indices = [i for i, x in enumerate(individual) if x > 0]
    zero_indices = [i for i, x in enumerate(individual) if x == 0]

    if len(non_zero_indices) > NUM_LIMIT:
        selected_indices = random.sample(non_zero_indices, NUM_LIMIT)
    else:
        selected_indices = non_zero_indices + random.sample(zero_indices, NUM_LIMIT - len(non_zero_indices))
    
    for i in range(len(individual)):
        if i not in selected_indices:
            individual[i] = 0
    
    if sum(individual) == 0:
        individual = create_individual()
        return individual

    n = SUM_CONSTRAINT / sum(individual)
    for i in range(len(individual)):
        individual[i] = int(individual[i] * n)

    d = SUM_CONSTRAINT - sum(individual)

    if selected_indices:
        chosen_index = random.choice(selected_indices)
        individual[chosen_index] += d

    return individual

def create_individual():
    individual = [0] * INDS_LEN

    chosen_positions = random.sample(range(INDS_LEN), NUM_LIMIT)

    remaining = SUM_CONSTRAINT
    for i in range(NUM_LIMIT-1):
        individual[chosen_positions[i]] = random.randint(1, remaining - (NUM_LIMIT - (i + 1)))
        remaining -= individual[chosen_positions[i]]
    individual[chosen_positions[-1]] = remaining

    return individual

def custom_cx(ind1, ind2):
    size = min(len(ind1), len(ind2))
    cxpoint = random.randint(1, size - 1)
    
    for i in range(cxpoint):
        ind1[i] = (ind1[i] + ind2[i]) / 2
        ind1[i] = int(ind1[i])

    for i in range(cxpoint, size):
        ind2[i] = (ind1[i] + ind2[i]) / 2
        ind2[i] = int(ind2[i])
    
    return ind1, ind2


In [16]:
# Setup DEAP
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=1, up=100, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Decorate evaluation function with feasibility check
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 0, repair))

# Evolutionary Algorithm
def main():
    pop = toolbox.population(n=20000)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("max", max)

    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=500, stats=stats, halloffame=hof)

    return hof[0]

# Run the GA
best_ind = main()

gen	nevals	max       
0  	20000 	(37026.0,)
1  	12011 	(37026.0,)
2  	12240 	(37026.0,)
3  	12032 	(37026.0,)
4  	12086 	(37026.0,)
5  	11907 	(37026.0,)
6  	12061 	(37026.0,)
7  	12011 	(37026.0,)
8  	11959 	(37026.0,)
9  	12147 	(37026.0,)
10 	11920 	(37026.0,)
11 	12056 	(37026.0,)
12 	12011 	(37026.0,)
13 	11897 	(37026.0,)
14 	12005 	(37026.0,)
15 	12024 	(37026.0,)
16 	12049 	(37026.0,)
17 	11918 	(37026.0,)
18 	12107 	(37026.0,)
19 	12056 	(37026.0,)
20 	11932 	(37026.0,)
21 	11889 	(37026.0,)
22 	12169 	(37026.0,)
23 	11960 	(37026.0,)
24 	12061 	(37026.0,)
25 	11952 	(37026.0,)
26 	11851 	(37026.0,)
27 	11893 	(37026.0,)
28 	12023 	(37026.0,)
29 	12030 	(37026.0,)
30 	12107 	(37026.0,)
31 	12092 	(37026.0,)
32 	11935 	(37026.0,)
33 	11955 	(37026.0,)
34 	12062 	(37026.0,)
35 	11967 	(37026.0,)
36 	11983 	(37026.0,)
37 	11982 	(37026.0,)
38 	12011 	(37026.0,)
39 	12086 	(37026.0,)
40 	11981 	(37026.0,)
41 	12083 	(37026.0,)
42 	12050 	(37026.0,)
43 	11998 	(37026.0,)
44 	11976 

In [17]:
for i in range(0, len(best_ind), int(len(best_ind)/NUM_LIMIT)):
    print(best_ind[i:i + int(len(best_ind)/NUM_LIMIT)])
    
lst = [x for x in best_ind if x !=0]
print("Best Individual:", lst, "Fitness:", best_ind.fitness.values[0])

[33]
[34]
[0]
[33]
Best Individual: [33, 34, 33] Fitness: 37026.0
